**1st part: Install Spark**

In [2]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
# Unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
# Set your spark folder to your system path environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [4]:
# Install findspark using pip
!pip install -q findspark

In [5]:
# Spark for Python
!pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8f1d1080c9e7475d0660b4990bd78988c1c08c019adaa84a92ac0104bfc04dbe
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [6]:
import findspark 
findspark.init()

In [8]:
from pyspark.sql import SparkSession

In [9]:
sc = SparkSession.builder.master("local[*]").getOrCreate()

**Second part: Load the data set from my desktop**

In [ ]:
files.upload() 

In [10]:
!ls

cities.csv  groups_topics.csv  spark-3.0.0-bin-hadoop3.2	topics.csv
events.csv  members.csv        spark-3.0.0-bin-hadoop3.2.tgz	venues.csv
groups.csv  sample_data        spark-3.0.0-bin-hadoop3.2.tgz.1


In [12]:
data_cities = sc.read.csv ("cities.csv", inferSchema= True , header= True) 

In [13]:
data_events = sc.read.csv ("events.csv", inferSchema= True , header= True) 

In [14]:
data_groups = sc.read.csv ("groups.csv", inferSchema= True , header= True) 

In [15]:
data_groupsTopics = sc.read.csv ("groups_topics.csv", inferSchema= True , header= True) 

In [16]:
data_members = sc.read.csv ("members.csv", inferSchema= True , header= True) 

In [19]:
data_topics = sc.read.csv ("/content/topics.csv", inferSchema= True , header= True) 

In [21]:
data_venues = sc.read.csv ("venues.csv", inferSchema= True , header= True) 

**Third part: discovering data**

In [22]:
data_cities.printSchema() #to ensure if spark read correctly the data set
data_cities.describe().toPandas().transpose()#use describe and convert the output to Pandas and use transpose to make it more clear 
                                     #in order to use some statistic values for better understanding and identify any anomalies
                                     #statistics are calculated by the describe() function on a dataset in PySpark (mean, max , min...)

root
 |-- city: string (nullable = true)
 |-- city_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- localized_country_name: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- member_count: integer (nullable = true)
 |-- ranking: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
city,13,None,None,Chicago,West New York
city_id,13,55304.53846153846,30251.68028009735,7093,95712
country,13,None,None,us,us
distance,13,1523.7936153846158,894.8365915047038,25.883,2526.837
latitude,13,41.29769223076922,2.3311693700915854,37.650002,46.689999
localized_country_name,13,None,None,USA,USA
longitude,13,-93.15230738461538,17.606069054513878,-122.43,-73.989998
member_count,13,29411.615384615383,66629.94457641707,4,229371
ranking,13,53.76923076923077,59.558310148058325,0,178


In [23]:
data_events.printSchema()
data_events.describe().toPandas().transpose()

root
 |-- event_id: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- event_url: string (nullable = true)
 |-- fee.accepts: string (nullable = true)
 |-- fee.amount: double (nullable = true)
 |-- fee.currency: string (nullable = true)
 |-- fee.description: string (nullable = true)
 |-- fee.label: string (nullable = true)
 |-- fee.required: integer (nullable = true)
 |-- group.created: string (nullable = true)
 |-- group.group_lat: double (nullable = true)
 |-- group.group_lon: double (nullable = true)
 |-- group_id: integer (nullable = true)
 |-- group.join_mode: string (nullable = true)
 |-- group.name: string (nullable = true)
 |-- group.urlname: string (nullable = true)
 |-- group.who: string (nullable = true)
 |-- headcount: integer (nullable = true)
 |-- how_to_find_us: string (nullable = true)
 |-- maybe_rsvp_count: integer (nullable = true)
 |-- event_name: string (nullable = t

,0,1,2,3,4
summary,count,mean,stddev,min,max
event_id,5807,2.432695579964476E8,5073073.18834785,153868222,zzsxdlyxnbtb
created,5807,None,None,2010-10-02 21:26:11,2017-10-25 22:21:34
description,5795,None,None,(30+ RSVPs so far https://sbdecin.splashthat.c...,~ FREE WEBINAR ~ In order to receive the Webin...
duration,5807,10769.002927501291,28220.28444406343,2700,1209600
event_url,5807,None,None,https://www.meetup.com/1001-Books-SF/events/24...,https://www.meetup.com/www-womensprosperitynet...
fee.accepts,5807,None,None,cash,wepay
fee.amount,5807,3.6688066127088,11.840787396296024,0.0,95.0
fee.currency,5807,None,None,USD,not_found
fee.description,5807,None,None,per person,per person


In [24]:
data_groups.printSchema()
data_groups.describe().toPandas().transpose()

root
 |-- group_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category.name: string (nullable = true)
 |-- category.shortname: string (nullable = true)
 |-- city_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- group_photo.base_url: string (nullable = true)
 |-- group_photo.highres_link: string (nullable = true)
 |-- group_photo.photo_id: integer (nullable = true)
 |-- group_photo.photo_link: string (nullable = true)
 |-- group_photo.thumb_link: string (nullable = true)
 |-- group_photo.type: string (nullable = true)
 |-- join_mode: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- link: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- group_name: string (nullable = true)
 |-- organizer.member_id: integer (nullable = true)
 |-- organizer.name:

,0,1,2,3,4
summary,count,mean,stddev,min,max
group_id,16330,1.5533966074709125E7,8812990.24423391,6388,26378470
category_id,16330,19.603000612369872,12.373332128736868,1,36
category.name,16330,None,None,LGBT,writing
category.shortname,16330,None,None,arts-culture,writing
city_id,16330,43408.562890385794,36927.85005299721,7093,94101
city,16330,None,None,Chicago,West New York
country,16330,None,None,US,US
created,16330,None,None,2002-10-08 17:22:11,2017-10-25 05:08:23
description,16328,None,None,! - - Midtown. : Wine & cheese ...,~~~ What your Meetup Group is about:\nJoin us ...


In [25]:
data_groupsTopics.printSchema()
data_groupsTopics.describe().toPandas().transpose()

root
 |-- topic_id: integer (nullable = true)
 |-- topic_key: string (nullable = true)
 |-- topic_name: string (nullable = true)
 |-- group_id: integer (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
topic_id,31212,23627.311771113673,72216.63950699117,83,1514434
topic_key,31212,None,None,20s-30s-social,youth-outdoor-education
topic_name,31212,None,None,20's & 30's Social,write
group_id,31212,1.519229923490965E7,8964537.806022385,6388,26378470


In [26]:
data_members.printSchema()
data_members.describe().toPandas().transpose()

root
 |-- member_id: integer (nullable = true)
 |-- bio: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- hometown: string (nullable = true)
 |-- joined: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- link: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- member_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- member_status: string (nullable = true)
 |-- visited: string (nullable = true)
 |-- group_id: integer (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
member_id,134868,1962362.3070187145,922994.7202380324,3,3164984
bio,134863,3.25,1.7677669529663689,!,~[masked]2011~WebbtraxRubymruby^^;; #startup
city,134868,None,None,Chicago,West New York
country,134868,None,None,us,us
hometown,134849,94332.46296296296,12205.924846059765,-,utica
joined,134867,None,None,2002-06-16 17:10:32,2017-11-09 15:39:25
lat,134867,40.33500937960494,1.362873900988254,37.61,42.01
link,134867,None,None,http://www.meetup.com/members/1000015,http://www.meetup.com/members/999921
lon,134867,-86.10917756016715,19.040619863590397,-122.51,-73.92


In [27]:
data_topics.printSchema()
data_topics.describe().toPandas().transpose()

root
 |-- topic_id: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- link: string (nullable = true)
 |-- members: integer (nullable = true)
 |-- topic_name: string (nullable = true)
 |-- urlkey: string (nullable = true)
 |-- main_topic_id: integer (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
topic_id,2509,118549.33359904344,265078.31565299357,83,1515654
description,2509,None,None,Are you a Knitter who wants to learn embroider...,not_found
link,2509,None,None,http://20s-30s-social.meetup.com/,http://zumba-wellness.meetup.com/
members,2509,125568.71821442805,945687.9915579193,0,25318915
topic_name,2509,None,None,20's & 30's Social,write
urlkey,2509,None,None,20s-30s-social,zumba-wellness
main_topic_id,2509,11254.623355918693,10351.523131981216,223,53052


In [28]:
data_venues.printSchema()
data_venues.describe().toPandas().transpose()

root
 |-- venue_id: integer (nullable = true)
 |-- address_1: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- localized_country_name: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- venue_name: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- rating_count: double (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- normalised_rating: double (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
venue_id,107093,1.5442661460777082E7,9790935.807771338,5286,25594988
address_1,107091,4569406.135802469,2.329223685602843E7,"! Battery Park Plaza,24 State Street(Between P...",zoom 740-571-690
city,107093,None,None,(Not necessarily) San Francisco,west chicago
country,107093,None,None,US,us
distance,107093,0.0,0.0,0.0,0.0
lat,107093,38.47720467961378,8.624403919627765,-45.100449,64.25995
localized_country_name,107093,None,None,USA,USA
lon,107093,-84.3384349957889,26.668502712548403,-123.082466,170.969421
venue_name,107075,8.309859154929577E58,7.002011783343734E59,# 1 TRAIN STATION ON 207TH STREET,{Q} Cloud Innovation Hub


**Fourth part: pre-cleaning**

In [33]:
data_cities.show()

+-------------------+-------+-------+--------+---------+----------------------+-----------+------------+-------+-----+-----+
|               city|city_id|country|distance| latitude|localized_country_name|  longitude|member_count|ranking|state|  zip|
+-------------------+-------+-------+--------+---------+----------------------+-----------+------------+-------+-----+-----+
|      West New York|   7093|     us|2524.541|40.790001|                   USA| -74.010002|         661|     32|   NJ| 7093|
|           New York|  10001|     us|2526.837|    40.75|                   USA| -73.989998|      229371|      0|   NY|10001|
|     New York Mills|  13417|     us|2392.162|43.099998|                   USA| -75.290001|          22|    109|   NY|13417|
|       East Chicago|  46312|     us|1810.371|41.639999|                   USA| -87.459999|          31|     90|   IN|46312|
|     New York Mills|  56567|     us|1418.834|46.689999|                   USA| -95.349998|           5|      1|   MN|56567|


In [34]:
print ((data_cities.count() , len (data_cities.columns)))
#Drop not performed

(13, 11)


In [35]:
#Events
print ((data_events.count() , len (data_events.columns)))

(5807, 48)


In [36]:
data_events.show()

+---------+-------------------+--------------------+--------+--------------------+-----------+----------+------------+---------------+---------+------------+-------------------+---------------+---------------+--------+---------------+--------------------+--------------------+-------------+---------+--------------------+----------------+--------------------+---------+--------------+------------+----------+------------+-------------------+-------------------+----------+--------------------+-----------------+-------------+-------------+--------+---------+----------------------------+-----------+--------------------+-----------+--------------+-----------+---------+----------+--------------+---------+--------------+
| event_id|            created|         description|duration|           event_url|fee.accepts|fee.amount|fee.currency|fee.description|fee.label|fee.required|      group.created|group.group_lat|group.group_lon|group_id|group.join_mode|          group.name|       group.urlname|   

In [40]:
#Remove some columns
data_events = data_events.drop("event_url","photo_url" )

In [41]:
data_events.show()

+---------+-------------------+--------------------+--------+-----------+----------+------------+---------------+---------+------------+-------------------+---------------+---------------+--------+---------------+--------------------+--------------------+-------------+---------+--------------------+----------------+--------------------+--------------+------------+----------+------------+-------------------+-------------------+----------+--------------------+-----------------+-------------+-------------+--------+---------+----------------------------+-----------+--------------------+-----------+--------------+-----------+---------+----------+--------------+---------+--------------+
| event_id|            created|         description|duration|fee.accepts|fee.amount|fee.currency|fee.description|fee.label|fee.required|      group.created|group.group_lat|group.group_lon|group_id|group.join_mode|          group.name|       group.urlname|    group.who|headcount|      how_to_find_us|maybe_rsvp

In [42]:
print ((data_events.count() , len (data_events.columns)))

(5807, 46)


In [43]:
#Groups
print ((data_groups.count() , len (data_groups.columns)))

(16330, 36)


In [44]:
data_groups.show()

+--------+-----------+--------------------+--------------------+-------+--------+-------+-------------------+--------------------+--------------------+------------------------+--------------------+----------------------+----------------------+----------------+---------+---------+--------------------+----------+-------+--------------------+-------------------+--------------------+------------------------+----------------------------+------------------------+--------------------------+--------------------------+--------------------+------+-----+----------+--------------------+----------+--------------+--------------------+
|group_id|category_id|       category.name|  category.shortname|city_id|    city|country|            created|         description|group_photo.base_url|group_photo.highres_link|group_photo.photo_id|group_photo.photo_link|group_photo.thumb_link|group_photo.type|join_mode|      lat|                link|       lon|members|          group_name|organizer.member_id|      orga

In [45]:
data_groups= data_groups.drop ("group_photo.base_url","group_photo.highres_link","group_photo.photo_link","group_photo.thumb_link", "link","lon","organizer.photo.base_url","organizer.photo.highres_link","organizer.photo.photo_link","organizer.photo.thumb_link")

In [46]:
print ((data_groups.count() , len (data_groups.columns)))

(16330, 26)


In [47]:
data_groups.show()

+--------+-----------+--------------------+--------------------+-------+--------+-------+-------------------+--------------------+--------------------+----------------+---------+---------+-------+--------------------+-------------------+--------------------+------------------------+--------------------+------+-----+----------+--------------------+----------+--------------+--------------------+
|group_id|category_id|       category.name|  category.shortname|city_id|    city|country|            created|         description|group_photo.photo_id|group_photo.type|join_mode|      lat|members|          group_name|organizer.member_id|      organizer.name|organizer.photo.photo_id|organizer.photo.type|rating|state|  timezone|             urlname|utc_offset|    visibility|                 who|
+--------+-----------+--------------------+--------------------+-------+--------+-------+-------------------+--------------------+--------------------+----------------+---------+---------+-------+----------

In [48]:
#groupsTopics
print ((data_groupsTopics.count() , len (data_groupsTopics.columns)))

(31212, 4)


In [49]:
data_groupsTopics.show()

+--------+----------+----------+--------+
|topic_id| topic_key|topic_name|group_id|
+--------+----------+----------+--------+
|      83|sportsfans|Sports Fan|  241031|
|      83|sportsfans|Sports Fan|  289172|
|      83|sportsfans|Sports Fan|  295444|
|      83|sportsfans|Sports Fan| 1040320|
|      83|sportsfans|Sports Fan| 1403055|
|      83|sportsfans|Sports Fan| 1426482|
|      83|sportsfans|Sports Fan| 1491757|
|      83|sportsfans|Sports Fan| 1492775|
|      83|sportsfans|Sports Fan| 1621920|
|      83|sportsfans|Sports Fan| 1683009|
|      83|sportsfans|Sports Fan| 1714061|
|      83|sportsfans|Sports Fan| 1733916|
|      83|sportsfans|Sports Fan| 3546292|
|      83|sportsfans|Sports Fan| 3680352|
|      83|sportsfans|Sports Fan| 3785922|
|      83|sportsfans|Sports Fan| 3884992|
|      83|sportsfans|Sports Fan| 4021122|
|      83|sportsfans|Sports Fan| 4274282|
|      83|sportsfans|Sports Fan| 4286742|
|      83|sportsfans|Sports Fan| 4316032|
+--------+----------+----------+--

In [50]:
#members
print ((data_members.count() , len (data_members.columns)))

(134868, 14)


In [51]:
data_members.show()

+---------+--------------------+--------+-------+------------+-------------------+-----+--------------------+-----+---------------+-----+-------------+-------------------+--------+
|member_id|                 bio|    city|country|    hometown|             joined|  lat|                link|  lon|    member_name|state|member_status|            visited|group_id|
+---------+--------------------+--------+-------+------------+-------------------+-----+--------------------+-----+---------------+-----+-------------+-------------------+--------+
|        3|           not_found|New York|     us|New York, NY|2007-05-01 22:04:37|40.72|http://www.meetup...|-74.0|    Matt Meeker|   NY|       active|2009-09-18 18:32:23|  490552|
|        3|           not_found|New York|     us|New York, NY|2011-01-23 14:13:17|40.72|http://www.meetup...|-74.0|    Matt Meeker|   NY|       active|2011-03-20 01:02:11| 1474611|
|        3|Hi, I'm Matt. I'm...|New York|     us|New York, NY|2010-12-30 18:47:34|40.72|http://

In [52]:
 data_members= data_members.drop ("link","lon")

In [53]:
data_members.show()

+---------+--------------------+--------+-------+------------+-------------------+-----+---------------+-----+-------------+-------------------+--------+
|member_id|                 bio|    city|country|    hometown|             joined|  lat|    member_name|state|member_status|            visited|group_id|
+---------+--------------------+--------+-------+------------+-------------------+-----+---------------+-----+-------------+-------------------+--------+
|        3|           not_found|New York|     us|New York, NY|2007-05-01 22:04:37|40.72|    Matt Meeker|   NY|       active|2009-09-18 18:32:23|  490552|
|        3|           not_found|New York|     us|New York, NY|2011-01-23 14:13:17|40.72|    Matt Meeker|   NY|       active|2011-03-20 01:02:11| 1474611|
|        3|Hi, I'm Matt. I'm...|New York|     us|New York, NY|2010-12-30 18:47:34|40.72|    Matt Meeker|   NY|       active|2011-01-18 20:37:23| 1490492|
|        3|Hi, I'm Matt. I'm...|New York|     us|New York, NY|2011-01-03 14:

In [54]:
print ((data_members.count() , len (data_members.columns)))

(134868, 12)


In [55]:
#topics
print ((data_topics.count() , len (data_topics.columns)))

(2509, 7)


In [56]:
data_topics.show()

+--------+--------------------+--------------------+-------+--------------------+----------+-------------+
|topic_id|         description|                link|members|          topic_name|    urlkey|main_topic_id|
+--------+--------------------+--------------------+-------+--------------------+----------+-------------+
|      83|Meet with others ...|http://sportsfans...| 471594|          Sports Fan|sportsfans|        10451|
|     130|Meet with Latin M...|http://latinmusic...| 759757|         Latin Music|latinmusic|        15018|
|     182|Want to practice ...|http://esl.meetup...|3176752|English as a Seco...|       esl|        10454|
|     183|Meet local Spanis...|http://spanish.me...|1618673|    Spanish Language|   spanish|        10454|
|     184|Meet and mingle w...|http://italian.me...| 465231|    Italian Language|   italian|        10454|
|     185|Meet local French...|http://french.mee...|1409448|     French Language|    french|        10454|
|     206|Meet other local ...|http:/

In [57]:
data_topics=data_topics.drop("link",)

In [59]:
print ((data_topics.count() , len (data_topics.columns)))
data_topics.show()

(2509, 6)
+--------+--------------------+-------+--------------------+----------+-------------+
|topic_id|         description|members|          topic_name|    urlkey|main_topic_id|
+--------+--------------------+-------+--------------------+----------+-------------+
|      83|Meet with others ...| 471594|          Sports Fan|sportsfans|        10451|
|     130|Meet with Latin M...| 759757|         Latin Music|latinmusic|        15018|
|     182|Want to practice ...|3176752|English as a Seco...|       esl|        10454|
|     183|Meet local Spanis...|1618673|    Spanish Language|   spanish|        10454|
|     184|Meet and mingle w...| 465231|    Italian Language|   italian|        10454|
|     185|Meet local French...|1409448|     French Language|    french|        10454|
|     206|Meet other local ...|2054003| Digital Photography|    digcam|          223|
|     223|Meet fellow shutt...|3277025|         Photography|     photo|          223|
|     225|Meet other local ...|  31895|     

In [60]:
#venues
print ((data_venues.count() , len (data_venues.columns)))

(107093, 14)


In [61]:
data_venues.show()
#Drop not performed

+--------+--------------------+--------+-------+--------+---------+----------------------+----------+--------------------+------+------------+-----+-----+-----------------+
|venue_id|           address_1|    city|country|distance|      lat|localized_country_name|       lon|          venue_name|rating|rating_count|state|  zip|normalised_rating|
+--------+--------------------+--------+-------+--------+---------+----------------------+----------+--------------------+------+------------+-----+-----+-----------------+
|    5286|      424 Park Ave S|New York|     us|     0.0|40.744259|                   USA|-73.983749|    Starbucks Coffee|   4.0|        51.0|   NY|10016|             3.92|
|    5293|        Union Square|New York|     us|     0.0| 40.73139|                   USA|  -73.9884|Virgin Megastore ...|  2.83|       109.0|   NY|10003|              2.8|
|    8356|   141 West 72nd St.|New York|     us|     0.0|40.778275|                   USA|-73.980095|Krispy Kreme Doug...|  2.11|      

**Fith part: Join method** 

In [89]:
def join(df1, df2, cond, how='left'):
    df = df1.join(df2, cond, how=how)
    repeated_columns = [c for c in df1.columns if c in df2.columns]
    for col in repeated_columns:
        df = df.drop(df2[col])
    return df

In [101]:
DF1=join(data_cities, data_groups,cond= ['city_id'],how = 'left')

In [102]:
DF1.show()

+-------+--------------+-------+--------+---------+----------------------+-----------+------------+-------+-----+-----+--------+-----------+--------------------+------------------+-------------------+--------------------+--------------------+----------------+---------+---------+-------+--------------------+-------------------+--------------------+------------------------+--------------------+------+----------+--------------------+----------+--------------+--------------------+
|city_id|          city|country|distance| latitude|localized_country_name|  longitude|member_count|ranking|state|  zip|group_id|category_id|       category.name|category.shortname|            created|         description|group_photo.photo_id|group_photo.type|join_mode|      lat|members|          group_name|organizer.member_id|      organizer.name|organizer.photo.photo_id|organizer.photo.type|rating|  timezone|             urlname|utc_offset|    visibility|                 who|
+-------+--------------+-------+----

In [103]:
DF2=join(data_events, data_venues, cond=['venue_id'], how ='left')

In [104]:
DF2.show()

+--------+------------+-------------------+--------------------+--------+-----------+----------+------------+---------------+---------+------------+-------------------+---------------+---------------+--------+---------------+--------------------+--------------------+--------------+---------+--------------------+----------------+--------------------+--------------+------------+----------+------------+-------------------+-------------------+----------+------------------+---------------+----------+-------------+---------+----------------------------+----------+---------------+-----------+--------------+-----------+---------+----------+--------------+---------+--------------+------------------+-------+-------+--------+---------+----------------------+----------+---------------+------+------------+-----+-----+-----------------+
|venue_id|    event_id|            created|         description|duration|fee.accepts|fee.amount|fee.currency|fee.description|fee.label|fee.required|      group.creat

In [106]:
DF3=join(data_groupsTopics, data_topics, cond=['topic_id'], how ='left')

In [107]:
DF3.show()

+--------+----------+----------+--------+--------------------+-------+----------+-------------+
|topic_id| topic_key|topic_name|group_id|         description|members|    urlkey|main_topic_id|
+--------+----------+----------+--------+--------------------+-------+----------+-------------+
|      83|sportsfans|Sports Fan|  241031|Meet with others ...| 471594|sportsfans|        10451|
|      83|sportsfans|Sports Fan|  289172|Meet with others ...| 471594|sportsfans|        10451|
|      83|sportsfans|Sports Fan|  295444|Meet with others ...| 471594|sportsfans|        10451|
|      83|sportsfans|Sports Fan| 1040320|Meet with others ...| 471594|sportsfans|        10451|
|      83|sportsfans|Sports Fan| 1403055|Meet with others ...| 471594|sportsfans|        10451|
|      83|sportsfans|Sports Fan| 1426482|Meet with others ...| 471594|sportsfans|        10451|
|      83|sportsfans|Sports Fan| 1491757|Meet with others ...| 471594|sportsfans|        10451|
|      83|sportsfans|Sports Fan| 1492775

In [108]:
DF4 = join(DF1,DF2, cond=['group_id'], how='left')

In [110]:
DF4.show()

+--------+-------+-------------+-------+--------+---------+----------------------+-----------+------------+-------+-----+-----+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+----------------+---------+---------+-------+--------------------+-------------------+-----------------+------------------------+--------------------+------+----------+--------------------+----------+--------------+-------------------+--------+--------+--------+-----------+----------+------------+---------------+---------+------------+-------------+---------------+---------------+---------------+----------+-------------+---------+---------+--------------+----------------+----------+--------------+------------+----------+------------+----------+-------+---------------+---------------+----------+-------------+---------+----------------------------+---------+----------+-----------+--------------+-----------+---------+--------------+----+-------------

In [111]:
DF5=join(DF4,DF3, cond=['group_id'], how='left')

In [112]:
DF5.show()

+--------+-------+-------------+-------+--------+---------+----------------------+-----------+------------+-------+-----+-----+-----------+------------------+------------------+-------------------+--------------------+--------------------+----------------+---------+---------+-------+--------------------+-------------------+-----------------+------------------------+--------------------+------+----------+--------------------+----------+--------------+-------------------+--------+--------+--------+-----------+----------+------------+---------------+---------+------------+-------------+---------------+---------------+---------------+----------+-------------+---------+---------+--------------+----------------+----------+--------------+------------+----------+------------+----------+-------+---------------+---------------+----------+-------------+---------+----------------------------+---------+----------+-----------+--------------+-----------+---------+--------------+----+--------------+--

In [113]:
DF_Batch= join(DF5, data_members, cond=['group_id'], how='left')

In [114]:
DF_Batch.show()

+--------+-------+-------+-------+--------+---------+----------------------+----------+------------+-------+-----+-----+-----------+--------------+------------------+-------------------+--------------------+--------------------+----------------+---------+---------+-------+--------------------+-------------------+--------------+------------------------+--------------------+------+----------+-------------+----------+----------+-------------------+--------+--------+--------+-----------+----------+------------+---------------+---------+------------+-------------+---------------+---------------+---------------+----------+-------------+---------+---------+--------------+----------------+----------+--------------+------------+----------+------------+----------+-------+---------------+---------------+----------+-------------+---------+----------------------------+---------+----------+-----------+--------------+-----------+---------+--------------+----+--------------+---------+----+----------+-

In [116]:
print ((DF_Batch.count() , len (DF_Batch.columns)))

(715728, 91)


In [117]:
DF_Batch.printSchema()
DF_Batch.describe().toPandas().transpose()

root
 |-- group_id: integer (nullable = true)
 |-- city_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- localized_country_name: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- member_count: integer (nullable = true)
 |-- ranking: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category.name: string (nullable = true)
 |-- category.shortname: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- group_photo.photo_id: integer (nullable = true)
 |-- group_photo.type: string (nullable = true)
 |-- join_mode: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- group_name: string (nullable = true)
 |-- organizer.member_id: 

,0,1,2,3,4
summary,count,mean,stddev,min,max
group_id,715724,4538732.679820713,6461622.9582241345,6388,26378470
city_id,715728,47072.73769225181,35590.80183879407,7093,95712
city,715728,None,None,Chicago,West New York
country,715728,None,None,us,us
...,...,...,...,...,...
hometown,701706,95977.33903133903,9617.911658486142,-,utica
joined,701782,None,None,2002-06-16 17:10:32,2017-11-09 15:39:25
member_name,701764,NaN,NaN,*)Serafina(*,~Seleee~
member_status,701782,None,None,active,active
